In [1]:
import os
import tensorflow as tf

# univariate cnn example
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from keras.utils import Sequence
from keras.utils import to_categorical

In [2]:
training_dir = os.listdir('../Pp_Training_SetA/')
list_id = []
for file in training_dir:
    filename = os.fsdecode(file)
    id = filename.lstrip("p").rstrip('.csv')
    list_id.append(id)

In [3]:
df = pd.DataFrame()
for ID in list_id:
    with open(f'../Pp_Training_SetA/p{ID}.csv', 'r') as f:
        df_id = pd.read_csv(f)
        df = df.append(df_id)
df.head()

/var/folders/wq/mnv64xbs77b03n4b7s0w2w3h0000gn/T/ipykernel_1294/174141211.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_id)
/var/folders/wq/mnv64xbs77b03n4b7s0w2w3h0000gn/T/ipykernel_1294/174141211.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_id)
/var/folders/wq/mnv64xbs77b03n4b7s0w2w3h0000gn/T/ipykernel_1294/174141211.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_id)
/var/folders/wq/mnv64xbs77b03n4b7s0w2w3h0000gn/T/ipykernel_1294/174141211.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_id)
/var/folders/wq/mnv64xbs77b03n4b7s0w2w3h0000gn/T/ipy

In [191]:
# Parameters
# Labels Pandas Dataframe column name
params = {'data_dir': '../Pp_Training_SetA/', 'list_IDs': list_id, 'labels': 'SepsisLabel', 'batch_size': 64, 'n_classes': 6, 'n_channels': 1, 'shuffle': True }


In [199]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, data_dir: str, list_IDs, labels, batch_size=32, dim=(40,39),
    n_channels=1, n_classes=2, shuffle=True):
        'Initialization'
        self.data_dir = data_dir
        self.list_IDs = list_IDs
        self.labels = labels
        self.batch_size = batch_size
        self.dim = dim
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
    

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))
    
    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        print(X)
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
    

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size), dtype=int)

        # X = np.empty((self.batch_size))
        # y = np.empty((self.batch_size))

        # Generate data
        for i, ID in enumerate(list_IDs_temp):

            # open file
            df = pd.DataFrame()
            with open(self.data_dir + 'p' + ID + ".csv", 'r') as f:
                df = pd.read_csv(f)

            seq = df.drop(labels=self.labels, axis=1)
            seq = seq.to_numpy()
            label = df[self.labels].to_numpy()
            
            # # Store sample
            np.append(X, seq)

            # # Store class
            np.append(y, to_categorical(label, num_classes=self.n_classes))
        
        print(X)
        print(y)

        return X, y


In [200]:
# training = DataGenerator(data_dir=training_dir, labels="SepsisLabel", list_IDs=list_id, batch_size=64, n_classes=2, n_channels=1, shuffle=True, dim=(40, 39), **params)
training = DataGenerator(**params)
# validation_generator = DataGenerator( **params)

# Design model
model = Sequential()
# Architecture
model.compile()
model.fit_generator(training,
                    use_multiprocessing=True,
                    workers=6)

[[[0.00000000e+000 2.16002568e-314 1.17797602e-318 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  ...
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]]

 [[0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  [0.00000000e+000 0.00000000e+000 0.00000000e+000 ... 0.00000000e+000
   0.00000000e+000 0.00000000e+000]
  ...
  [0.00000000e+000 0.00

/var/folders/wq/mnv64xbs77b03n4b7s0w2w3h0000gn/T/ipykernel_1445/4063549074.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(training,


AttributeError: 'NoneType' object has no attribute 'shape'